Decission Tree from Scratch. 

Decision trees are a non-parametric model used for both regression and classification tasks. Decision trees are constructed from only two elements – nodes and branches. 

types of nodes:
1. root node
2. internal/decision node
3. leaf node

Root node is determined using entropy. Entropy values ranges from 0 to 1. The entropy gives the purity of split like 0 being impure and 1 being pure. 

Next step we calculate the information gain which is the avergae of all the entropy values based ona specific split. 

This is done in a recursivly to get fit a best decision tree. 

The model is build on iris dataset.


Load dataset

In [ ]:
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()

X = iris['data']
y = iris['target']

Node class to build tree and store data in the nodes

In [ ]:
class Node:
    '''
    Helper class which implements a single tree node.
    '''
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

Decision tree class to implemnet classifier. 


In [ ]:
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        #calculates the impurity of an input vector
    def _entropy(s):
        counts = np.bincount(np.array(s, dtype=np.int64))
        percentages = counts / len(s)
        entropy = 0
        for pct in percentages:
            if pct > 0:
                entropy += pct * np.log2(pct)
        return -entropy
    
    # average of entropy for every split
    def _information_gain(self, parent, left_child, right_child):
        num_left = len(left_child) / len(parent)
        num_right = len(right_child) / len(parent)
        return self._entropy(parent) - (num_left * self._entropy(left_child) + num_right * self._entropy(right_child))
    # checks if the split is best
    def best_split(self, X, y):
        best_split = {}
        best_info_gain = -1
        n_rows, n_cols = X.shape
        
        for f_idx in range(n_cols):
            X_curr = X[:, f_idx]
            for threshold in np.unique(X_curr):
                df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
                df_left = np.array([row for row in df if row[f_idx] <= threshold])
                df_right = np.array([row for row in df if row[f_idx] > threshold])
                if len(df_left) > 0 and len(df_right) > 0:
                    y = df[:, -1]
                    y_left = df_left[:, -1]
                    y_right = df_right[:, -1]
                    gain = self._information_gain(y, y_left, y_right)
                    if gain > best_info_gain:
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gain': gain
                        }
                        best_info_gain = gain
        return best_split
    
    def _build(self, X, y, depth=0):
        n_rows, n_cols = X.shape
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            best = self.best_split(X, y)
            if best['gain'] > 0:
                left = self._build(
                    X=best['df_left'][:, :-1], 
                    y=best['df_left'][:, -1], 
                    depth=depth + 1
                )
                right = self._build(
                    X=best['df_right'][:, :-1], 
                    y=best['df_right'][:, -1], 
                    depth=depth + 1
                )
                return Node(
                    feature=best['feature_index'], 
                    threshold=best['threshold'], 
                    data_left=left, 
                    data_right=right, 
                    gain=best['gain']
                )
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )
    
    def fit(self, X, y):
        self.root = self._build(X, y)
        
    def predict_recusive(self, x, tree):
        if tree.value != None:
            return tree.value
        feature_value = x[tree.feature]
        
        # left
        if feature_value <= tree.threshold:
            return self.predict_recusive(x=x, tree=tree.data_left)
        
        # right
        if feature_value > tree.threshold:
            return self.predict_recusive(x=x, tree=tree.data_right)
        
    def predict(self, X):
        return [self.predict_recusive(x, self.root) for x in X]

Split data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

fit model using training data and preedict it for test data

In [ ]:
import numpy as np
from collections import Counter

model = DecisionTree()
model.fit(X_train, y_train)
preds = model.predict(X_test)

Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, preds)

1.0